In [0]:
# Upgrade Databricks SDK to the latest version and restart Python to see updated packages
%pip install --upgrade databricks-sdk==0.70.0
%restart_python

from databricks.sdk.service.jobs import JobSettings as Job

#xxx should be replaced by user id
Refresh_Silver_Movies_Job_Setup = Job.from_dict(
    {
        "name": "Refresh_Silver_Movies_Job_Setup",
        "tasks": [
            {
                "task_key": "Load_CSV_To_Bronze",
                "notebook_task": {
                    "notebook_path": "/Workspace/Users/xxx/Movie-Data-Analysis-Using-Databricks/Load Bronze - Movie/load data from csv to bronze layer",
                    "source": "WORKSPACE",
                },
            },
            {
                "task_key": "Setup_Genre_Master",
                "depends_on": [
                    {
                        "task_key": "Load_CSV_To_Bronze",
                    },
                ],
                "notebook_task": {
                    "notebook_path": "/Workspace/Users/xxx/Movie-Data-Analysis-Using-Databricks/Load Silver Tables - Movies/load silver genre master setup",
                    "source": "WORKSPACE",
                },
            },
            {
                "task_key": "Setup_Platform_Master",
                "depends_on": [
                    {
                        "task_key": "Setup_Genre_Master",
                    },
                ],
                "notebook_task": {
                    "notebook_path": "/Workspace/Users/xxx/Movie-Data-Analysis-Using-Databricks/Load Silver Tables - Movies/load silver platform master setup",
                    "source": "WORKSPACE",
                },
            },
            {
                "task_key": "Load_Silver_Movie_data",
                "depends_on": [
                    {
                        "task_key": "Setup_Platform_Master",
                    },
                ],
                "notebook_task": {
                    "notebook_path": "/Workspace/Users/xxx/Movie-Data-Analysis-Using-Databricks/Load Silver Tables - Movies/load silver movie data transaction",
                    "source": "WORKSPACE",
                },
            },
        ],
        "queue": {
            "enabled": True,
        },
        "performance_target": "PERFORMANCE_OPTIMIZED",
    }
)

from databricks.sdk import WorkspaceClient

w = WorkspaceClient()
#w.jobs.reset(new_settings=Refresh_Silver_Movies_Job_Setup, job_id=314417073838345)
w.jobs.create(**Refresh_Silver_Movies_Job_Setup.as_shallow_dict())
# or create a new job using: w.jobs.create(**Refresh_Silver_Movies.as_shallow_dict())
